In [1]:
!nvidia-smi

Sat Sep 25 19:21:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
pip install tensorflow==1.15.0 --quiet

     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 50 kB 9.0 MB/s 
     |████████████████████████████████| 503 kB 75.1 MB/s 
     |████████████████████████████████| 3.8 MB 61.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.13.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.


In [3]:
pip install tensorflow-hub==0.10.0 --quiet

     |████████████████████████████████| 107 kB 6.6 MB/s 


In [39]:
!pip install bert-tensorflow --quiet

In [40]:
import pandas as pd
import numpy as np
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
RANDOM_SEED = 42
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [41]:
BERT_VOCAB= '/content/drive/MyDrive/Formation_Machine_Learning/P7/BERT-Base_Uncased/vocab.txt'
BERT_INIT_CHKPNT = '/content/drive/MyDrive/Formation_Machine_Learning/P7/BERT-Base_Uncased/bert_model.ckpt'
BERT_CONFIG = '/content/drive/MyDrive/Formation_Machine_Learning/P7/BERT-Base_Uncased/bert_config.json'

In [42]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

## Data

In [9]:
data=pd.read_csv('/content/drive/MyDrive/Formation_Machine_Learning/P7/data/10label_data.csv')
data.drop(['Unnamed: 0','Id'], axis=1,inplace=True)
data['bodytitles']=data['Title']+','+data['Body']
data

,Title,Body,Tags,bodytitles
0,check property set blazor,"big issue property nullable , want check set ,...",csharp blazor,"check property set blazor,big issue property n..."
1,movaps vs. movups gcc decide,recently research segfault piece software comp...,cplusplus gplusplus optimization memoryalignment,"movaps vs. movups gcc decide,recently research..."
2,work match part regex,hundred fairly simple regular expression match...,python regex grammar textparsing,"work match part regex,hundred fairly simple re..."
3,compatible side side ndk version find react na...,create blank react native app add webrtc packa...,android reactjs reactnative webrtc,compatible side side ndk version find react na...
4,clear previous data infinite scroll,cod infinite scroll . user scroll load additio...,javascript infinitescroll,"clear previous data infinite scroll,cod infini..."
...,...,...,...,...
15217,ienumerator method calculate next result backg...,method return long calculation process record ...,csharp parallelprocessing ienumerable ienumera...,ienumerator method calculate next result backg...
15218,"imemorycache , refresh cache eviction",try migrate dotnet framework application dotne...,csharp asp.netcore caching .netcore memorycache,"imemorycache , refresh cache eviction,try migr..."
15219,unable load system.threading.tasks.extensions,web project build dotnet framework ... try add...,csharp nuget npgsql,"unable load system.threading.tasks.extensions,..."
15220,add new user loggedin user,weird behaviour come create new user scratchre...,csharp asp.netcoremvc entityframeworkcore asp....,"add new user loggedin user,weird behaviour com..."


In [10]:
def top_tags(df,max_tags):

  #BOW from the tags
  tag_vect=CountVectorizer()
  vectorized_tags=tag_vect.fit_transform(df['Tags'])

  #filter the top 100 tags according to occurences 
  tags_count=np.sum(vectorized_tags.toarray(),axis=0)
  sorted_tags=np.argsort(-tags_count)
  top_tags_index=sorted_tags[:max_tags]

  tags_dict={'tag':[],
             'tag_occurences':[]}

  for i in top_tags_index:
    tags_dict['tag'].append(tag_vect.get_feature_names()[i])
    tags_dict['tag_occurences'].append(tags_count[i])

  return pd.DataFrame(tags_dict)

In [11]:
LEN_TAGS=10
top_tag_dt=top_tags(data,LEN_TAGS)
top_tag_dt

,tag,tag_occurences
0,javascript,3118
1,python,2967
2,html,1958
3,csharp,1780
4,tensorflow,1669
5,arrays,1515
6,php,1513
7,cplusplus,1511
8,android,1323
9,css,1161


### Balance the data

In [12]:
titles=[] # To store the filtered clean_question values
bodies=[]
mix_data=[]
y=[] # to store the corresponding tags
# Convert to list data type
lst_top_tags = top_tag_dt['tag'].values.tolist()

for i in range(len(data['Tags'])):
    temp=[]
    for tag in data['Tags'].iloc[i].split():
        if tag in lst_top_tags:
            temp.append(tag)

    if(len(temp)>0):
        titles.append(data['Title'].iloc[i])
        bodies.append(data['Body'].iloc[i])
        mix_data.append(data['bodytitles'].iloc[i])
        y.append(temp)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
 
label_values = mlb.fit_transform(y)

### **Split the dataset**

### Split the question Titles

In [14]:
# First Split for Train and Test
x_train,x_test,y_train,y_test = train_test_split(titles, label_values, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x_train,x_val,y_train,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

In [15]:
train_dt=pd.concat([pd.DataFrame(x_train, columns=['titles']),pd.DataFrame(y_train, columns=mlb.classes_)], axis=1)
train_dt.reset_index(inplace=True)
val_dt=pd.concat([pd.DataFrame(x_val, columns=['titles']),pd.DataFrame(y_val, columns=mlb.classes_)], axis=1)
val_dt.reset_index(inplace=True)
test_dt=pd.concat([pd.DataFrame(x_test, columns=['titles']),pd.DataFrame(y_test, columns=mlb.classes_)], axis=1)
test_dt.reset_index(inplace=True)

In [51]:
train3_dt

,index,titles,android,arrays,cplusplus,csharp,css,html,javascript,php,python,tensorflow
0,0,"empty inside array e.g . empty,sorry mislead t...",0,1,0,0,0,0,1,0,0,0
1,1,"time complexity check membership dict.items,ti...",0,0,0,0,0,0,0,0,1,0
2,2,"add bearer token retrofit request javascript,h...",0,0,0,0,0,0,1,0,0,0
3,3,"serilog , microsoft.extensions.logging autofac...",0,0,0,1,0,0,0,0,0,0
4,4,"difference this.function function.callthis,rea...",0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9027,9027,omnisharp server throw system.io.filenotfounde...,0,0,0,1,0,0,0,0,0,0
9028,9028,"tf.print work,code snippet two statement funct...",0,0,0,0,0,0,0,0,1,1
9029,9029,"string comparison regardless php order,want kn...",0,1,0,0,0,0,0,1,0,0
9030,9030,fortran interface call clang function return p...,0,1,0,0,0,0,0,0,0,0


### Split the question Bodies

In [16]:
# First Split for Train and Test
x2_train,x2_test,y2_train,y2_test = train_test_split(bodies, label_values, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x2_train,x2_val,y2_train,y2_val = train_test_split(x2_train, y2_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

In [17]:
train2_dt=pd.concat([pd.DataFrame(x2_train, columns=['titles']),pd.DataFrame(y2_train, columns=mlb.classes_)], axis=1)
train2_dt.reset_index(inplace=True)
val2_dt=pd.concat([pd.DataFrame(x2_val, columns=['titles']),pd.DataFrame(y2_val, columns=mlb.classes_)], axis=1)
val2_dt.reset_index(inplace=True)
test2_dt=pd.concat([pd.DataFrame(x2_test, columns=['titles']),pd.DataFrame(y2_test, columns=mlb.classes_)], axis=1)
test2_dt.reset_index(inplace=True)

### Split the Body-titles data

In [18]:
# First Split for Train and Test
x3_train,x3_test,y3_train,y3_test = train_test_split(mix_data, label_values, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x3_train,x3_val,y3_train,y3_val = train_test_split(x3_train, y3_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

In [19]:
train3_dt=pd.concat([pd.DataFrame(x3_train, columns=['titles']),pd.DataFrame(y3_train, columns=mlb.classes_)], axis=1)
train3_dt.reset_index(inplace=True)
val3_dt=pd.concat([pd.DataFrame(x3_val, columns=['titles']),pd.DataFrame(y3_val, columns=mlb.classes_)], axis=1)
val3_dt.reset_index(inplace=True)
test3_dt=pd.concat([pd.DataFrame(x3_test, columns=['titles']),pd.DataFrame(y3_test, columns=mlb.classes_)], axis=1)
test3_dt.reset_index(inplace=True)

In [20]:
ID = 'id'
TITLE_COLUMN = 'Title'
BODY_COLUMN = 'Body'
MIX_COLUMN='bodytitles'
LABEL_COLUMNS = list(data.columns[2:])

In [21]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [35]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[2:]
        else:
            labels = [0]*10
            
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [46]:
train_examples = create_examples(train_dt)


In [47]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [25]:
MAX_SEQ_LENGTH = 128

In [48]:
features_examples=convert_examples_to_features(train_examples, MAX_SEQ_LENGTH, tokenizer)

empty inside array e.g . empty


UnparsedFlagAccessError: ignored

In [30]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [31]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [32]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature

In [33]:
def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()

In [34]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([10], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn

In [35]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [36]:
train_file = os.path.join('/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/Train_record', "train_10label.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [37]:
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 9032
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 282


In [38]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [39]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)

In [40]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn


In [41]:
OUTPUT_DIR = "/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/10_labels_Output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [42]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= 10,
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

In [43]:
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/10_labels_Output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f234ddd1090>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

In [45]:
eval_file = os.path.join('/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/Eval_record', "eval_10label.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(val2_dt)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [46]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:10;logits:Tensor("loss/BiasAdd:0", shape=(?, 10), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 10), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-09-24T09:54:08Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/10_labels_Output/model.ckpt-282
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-09-24-09:54:32
INFO:tensorflow:Saving dict for global step 282: 0 = 0.90128917, 1 = 0.9433319, 2 = 0.933332, 3 = 0.8670735, 4 = 0.96458864, 5 = 0.9354229, 6 = 0.80546147, 7 = 0.86335796, 8 = 0.8852989, 9 = 0.9669323, eval_loss = 0.23986562, global_step = 282, loss = 0.2399443

In [47]:

output_eval_file = os.path.join("/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/Eval_output", "eval_10label.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  0 = 0.90128917
INFO:tensorflow:  1 = 0.9433319
INFO:tensorflow:  2 = 0.933332
INFO:tensorflow:  3 = 0.8670735
INFO:tensorflow:  4 = 0.96458864
INFO:tensorflow:  5 = 0.9354229
INFO:tensorflow:  6 = 0.80546147
INFO:tensorflow:  7 = 0.86335796
INFO:tensorflow:  8 = 0.8852989
INFO:tensorflow:  9 = 0.9669323
INFO:tensorflow:  eval_loss = 0.23986562
INFO:tensorflow:  global_step = 282
INFO:tensorflow:  loss = 0.23994432


In [48]:
predict_examples = create_examples(test2_dt,False)

In [ ]:
test_features = convert_examples_to_features(predict_examples, MAX_SEQ_LENGTH, tokenizer)

In [50]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [51]:
print('Beginning Predictions!')
current_time = datetime.now()

predict_input_fn = input_fn_builder(features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
print("Prediction took time ", datetime.now() - current_time)

Beginning Predictions!
Prediction took time  0:00:00.001448


In [97]:
predict_input_fn

<function __main__.input_fn_builder.<locals>.input_fn>

In [93]:
predictions

<generator object Estimator.predict at 0x7f7221839d50>

In [52]:
LABEL_COLUMNS=train2_dt.columns[2:]
LABEL_COLUMNS

Index(['android', 'arrays', 'cplusplus', 'csharp', 'css', 'html', 'javascript',
       'php', 'python', 'tensorflow'],
      dtype='object')

In [53]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS

    return dff

In [54]:
output_df = create_output(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:10;logits:Tensor("loss/BiasAdd:0", shape=(?, 10), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 10), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 10), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/10_labels_Output/model.ckpt-282
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [55]:
output_df.to_csv("/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/test_pred_10label.csv", index=False)

In [48]:
import pandas as pd
import numpy as np
data=pd.read_csv('/content/drive/MyDrive/Formation_Machine_Learning/P7/Tensorflow_Models/Bodies/test_pred_10label.csv')
data.head()

,android,arrays,cplusplus,csharp,css,html,javascript,php,python,tensorflow
0,0.104040,0.040999,0.034940,0.134504,0.081128,0.166473,0.396555,0.181039,0.038766,0.012718
1,0.224098,0.027944,0.023905,0.055317,0.463838,0.687424,0.393773,0.103564,0.061748,0.035386
2,0.025106,0.068269,0.151203,0.074050,0.023477,0.035839,0.074082,0.029471,0.700893,0.325874
3,0.095022,0.034160,0.188090,0.558342,0.016996,0.029020,0.102755,0.135434,0.170838,0.064507
4,0.585575,0.027434,0.034997,0.112908,0.141208,0.256887,0.255748,0.244615,0.041419,0.038511


In [2]:
def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

In [5]:
#define candidate threshold values
threshold  = np.arange(0.3,0.61,0.01)
threshold

array([0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 ,
       0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51,
       0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ])

In [39]:
test_dt.drop(['index','titles'],axis=1).values.ravel() 

array([0, 0, 0, ..., 0, 0, 0])

In [49]:
from sklearn import metrics
scores=[] # Store the list of f1 scores for prediction on each threshold

#convert labels to 1D array
y_true = test2_dt.drop(['index','titles'],axis=1).values.ravel() 

for thresh in threshold:
    
    #classes for each threshold
    pred_bin_label = classify(data.values,thresh) 

    #convert to 1D array
    y_pred = np.array(pred_bin_label).ravel()

    scores.append(metrics.f1_score(y_true,y_pred))

In [50]:
opt_thresh = threshold[scores.index(max(scores))]
print(f'Optimal Threshold Value = {opt_thresh}')

Optimal Threshold Value = 0.37000000000000005


In [51]:
#predictions for optimal threshold
y_pred_labels = classify(data.values,opt_thresh)
y_pred = np.array(y_pred_labels).ravel() # Flatten

In [62]:
y_pred.shape

(12550,)

In [52]:
flat_true_labels=test_dt.drop(['index','titles'],axis=1).values.ravel() 
flat_true_labels.shape

(12550,)

In [53]:
from sklearn.metrics import accuracy_score,hamming_loss

accuracy_score(flat_true_labels ,y_pred)

0.9123505976095617

In [54]:
hamming_loss(flat_true_labels ,y_pred)

0.08764940239043825